<a href="https://colab.research.google.com/github/thanhyx4/H-M-Personalized-Fashion-Recommendations/blob/main/Data_understand_RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from math import sqrt
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()

from datetime import datetime, timedelta
import gc

In [ ]:
!nvidia-smi  #check GPU

In [ ]:
import sys
!cp ../input/rapids/rapids.21.06 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

import cudf

# Read the transactions data
##### article_id: number => dtypes: int32
##### customer_id: hex => change to int64 (reduct memory, enhance speed)
##### read, modify dataframe by cudf (GPU)

In [2]:
# df  = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',
#                             usecols= ['t_dat', 'customer_id', 'article_id'], 
#                             dtype={'article_id': 'int32', 't_dat': 'string', 'customer_id': 'string'})
# df ['customer_id'] = df ['customer_id'].str[-16:].str.hex_to_int().astype('int64')
    
# df['t_dat'] = cudf.to_datetime(df['t_dat'])
# last_ts = df['t_dat'].max()

#change -> int32 => need to add '0' in the first of article_id

In [2]:
df_article =pd.read_csv('/kaggle/input/dataset/articles (1).csv',dtype={'article_id': 'str'})
df_tran= pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',
                 usecols = ['t_dat', 'customer_id', 'article_id'],
                 dtype={'article_id': 'str'})
df_cus =pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

df_tran['t_dat'] = pd.to_datetime(df_tran['t_dat'])


In [3]:
df_grp_prod_type = df_tran.merge(df_article[['article_id','idxgrp_idx_prdtyp','is_for_male', 'is_for_female', 'is_for_mama']],\
                                          on='article_id', how='left')

NameError: name 'df' is not defined

# 1. Trending weekly

In [9]:
def trending_week(df_grp_prod_type, day_start, day_end):     #day_end: 2020-09-22 >= a > 2020-09-15
    day_start_last_week = '2020-' + day_start
    day_end_last_week = '2020-' + day_end
    day_start_last_week_last_year = '2019-' + day_start
    day_end_last_week_last_year = '2019-' + day_end
    
    df_tran_1w = df_grp_prod_type[ (df_grp_prod_type['t_dat'] > pd.to_datetime(day_start_last_week)) &  \
                                  (df_grp_prod_type['t_dat'] <=  pd.to_datetime(day_end_last_week))    ]
    df_tran_1w.reset_index(inplace=True)
    df_tran_1w_lastyear = df_grp_prod_type[ (df_grp_prod_type['t_dat'] > pd.to_datetime(day_start_last_week_last_year))  & \
                                  (df_grp_prod_type['t_dat'] < pd.to_datetime(day_end_last_week_last_year)) ] 

    
#     df_tran_1w_male = df_tran_1w[df_tran_1w['is_for_male'] == 1]
#     df_tran_1w_mama = df_tran_1w[df_tran_1w['is_for_mama'] == 1]
#     df_tran_1w_female = df_tran_1w[df_tran_1w['is_for_female'] == 1]
    
    return df_tran_1w['article_id'].value_counts().head(12)
    
    
trending_week(df_tran, '09-15', '09-22')
#df_tran_1w_last2year.to_csv("/kaggle/working/df_tran_1w_last2year.csv")


0924243001    852
0924243002    635
0918522001    609
0923758001    592
0866731001    552
0909370001    537
0751471001    526
0915529003    495
0915529005    491
0448509014    490
0762846027    474
0714790020    468
Name: article_id, dtype: int64

# 2. Popularity

Add the last day of billing week

In [20]:
# %%time       #check time GPU
# ##add the last day of billing week of each transactions (previous weeks from here 23-9-2020 Tuesday)

# #df['ldbw'] = df['t_dat'].progress_apply(lambda d: last_ts - (last_ts - d).floor('7D'))
# ##change -> day of week -> ldbw = t_dat - dow + 1 
# ##the last day of week = Tuesday -> dow >= 2 (>= Wednesday): ldbw + 7 days
# tmp = df[['t_dat']].copy().to_pandas()
# tmp['dow'] = tmp['t_dat'].dt.dayofweek
# tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['dow'] - 1, unit='D')
# tmp.loc[tmp['dow'] >=2 , 'ldbw'] = tmp.loc[tmp['dow'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['dow'] >=2])) * 7, unit='D')

# df['ldbw'] = tmp['ldbw'].values



tmp = df_tran[['t_dat']].copy()
tmp['dow'] = tmp['t_dat'].dt.dayofweek
tmp['ldbw'] = tmp['t_dat'] - pd.TimedeltaIndex(tmp['dow'] - 1, unit='D')
tmp.loc[tmp['dow'] >=2 , 'ldbw'] = tmp.loc[tmp['dow'] >=2 , 'ldbw'] + pd.TimedeltaIndex(np.ones(len(tmp.loc[tmp['dow'] >=2])) * 7, unit='D')

df_tran['ldbw'] = tmp['ldbw'].values



Count the number of transactions per week

In [21]:
weekly_sales = df_tran.drop('customer_id', axis=1).groupby(['ldbw', 'article_id']).count().reset_index()
weekly_sales = weekly_sales.rename(columns={'t_dat': 'count'})

df_tran = df_tran.merge(weekly_sales, on=['ldbw', 'article_id'], how = 'left')

Let's assume that in the target week sales will be similar to the last week of the training data

In [12]:
weekly_sales.head()

,ldbw,article_id,count
0,2018-09-25,0108775015,224
1,2018-09-25,0108775044,96
2,2018-09-25,0108775051,31
3,2018-09-25,0110065001,28
4,2018-09-25,0110065002,14


In [22]:
def popularity(df_tran, weekly_sales, day_end):  #day end of the week produce candidate
    day_end = pd.to_datetime(day_end)
    df = df_tran[df_tran['t_dat'] <= day_end].copy()
    weekly_sales_tmp = weekly_sales[weekly_sales['ldbw'] <= day_end].copy()  
    weekly_sales_tmp = weekly_sales_tmp.reset_index().set_index('article_id')
    df = df.merge(
        weekly_sales_tmp.loc[weekly_sales_tmp['ldbw']==day_end, ['count']],
        on='article_id', suffixes=("", "_targ"))
    #filter all article_id which purchased in the last week (in all history)
    df['count_targ'].fillna(0, inplace=True)
    #count_target: count in the last week (truong hop count 2 cai bang nhau ->quotient = 0?)
    del weekly_sales
    df['quotient'] = df['count_targ'] / df['count'] 
    #number transac each article_id in the last week / number transac each article_id each previous week

    target_sales = df.drop('customer_id', axis=1).groupby('article_id')['quotient'].sum()
    general_pred = target_sales.nlargest(20).index.tolist()
    del target_sales
    #target_sales.nlargest(20).to_csv("popularity.csv")
    return general_pred

-> quotient :
lượng mua tuần này/ lần lượt count  của từng tuần (trước) của article_id đó 


-> tổng: độ phổ biến tính đến tuần trước của món hàng đó #? những sản phẩm mới ra mắt tuần này

In [24]:
popularity(df_tran, weekly_sales, '2020-09-15')

['0448509014',
 '0706016001',
 '0751471001',
 '0715624001',
 '0706016002',
 '0673677002',
 '0158340001',
 '0706016003',
 '0685814003',
 '0573085028',
 '0685814001',
 '0372860001',
 '0372860002',
 '0579541001',
 '0562245046',
 '0610776002',
 '0678942001',
 '0156231001',
 '0706016006',
 '0568601006']

# 3. Repurchased rate

In [9]:
purchase_dict = {}

for i in tqdm(df.index):
    cust_id = df.at[i, 'customer_id']
    art_id = df.at[i, 'article_id']
    t_dat = df.at[i, 't_dat']

    if cust_id not in purchase_dict:
        purchase_dict[cust_id] = {}

    if art_id not in purchase_dict[cust_id]:
        purchase_dict[cust_id][art_id] = 0
    
    x = max(1, (last_ts - t_dat).days)

    a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
    y = a / np.sqrt(x) + b * np.exp(-c*x) - d

    value = df.at[i, 'quotient'] * max(0, y) # độ phổ biến tuần mua so với tuần này * tỉ lệ mua lại món hàng đó
    purchase_dict[cust_id][art_id] += value  #moi lan mua tang ti le mua lai mon do do ?

100%|██████████| 12541136/12541136 [16:46<00:00, 12459.09it/s]


In [38]:
a, b, c, d = 2.5e4, 1.5e5, 2e-1, 1e3
#y = a / np.sqrt(x) + b * np.exp(-c*x) - d

df_tran['xdays'] = (last_ts - df_tran['t_dat']).dt.days.astype(int)
df_tran['tmp1'] = 1
df_tran['xdays'] = df_tran[['xdays', 'tmp1']].max(axis=1)
df_tran['repurchased_rate'] = a / np.sqrt(df_tran['xdays']) + b * np.exp(-c*df_tran['xdays']) - d
df_tran['tmp2'] = 0
df_tran['repurchased_rate'] = df_tran[['repurchased_rate', 'tmp2']].max(axis=1)
df_tran.drop(columns = ['tmp1', 'tmp2'], inplace = True)
df_tran.tail()

,t_dat,customer_id,article_id,xdays,repurchased_rate,tpm2
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,0929511001,1,146809.612962,0
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,0891322004,1,146809.612962,0
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,0918325001,1,146809.612962,0
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,0833459002,1,146809.612962,0
31788323,2020-09-22,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0898573003,1,146809.612962,0


In [42]:
purchase_dict = {}

#df_tran
repurchased_pred = df_tran.groupby(['customer_id', 'article_id'])['repurchased_rate'].sum()
repurchased_pred.reset_index()
repurchased_pred.group

,customer_id,article_id,repurchased_rate
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0176209023,0.000000
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601006,268.046058
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043,10069.381120
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0607642008,212.678125
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0625548001,0.000000
...,...,...,...
27306434,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0821395003,917.412472
27306435,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0840360003,940.376314
27306436,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0866755002,1981.617767
27306437,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0882810001,1606.431707


# Submit

In [ ]:
sub = pd.read_csv(data_path / 'sample_submission.csv')

pred_list = []
for cust_id in tqdm(sub['customer_id']):
    if cust_id in purchase_dict:
        series = pd.Series(purchase_dict[cust_id])
        series = series[series > 0]
        l = series.nlargest(N).index.tolist()
        if len(l) < N:
            l = l + general_pred[:(N-len(l))]
    else:
        l = general_pred
    pred_list.append(' '.join(l))

sub['prediction'] = pred_list
sub.to_csv('submission.csv', index=None)

In [16]:
df_submission = pd.read_csv("/kaggle/input/dataset/df_submission.csv")


In [24]:
df_submission['prediction'] = " ".join(general_pred)
df_submission.to_csv(f'submission1.csv', index = False)